In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['MB']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('methylene blue', 926),
 ('medulloblastoma', 396),
 ('mushroom body', 176),
 ('microbubbles', 92),
 ('molecular beacon', 58),
 ('myocardial bridge', 43),
 ('mitochondrial biogenesis', 28),
 ('maneb', 26),
 ('myoblasts', 22),
 ('multibacillary', 22),
 ('major bleeding', 18),
 ('multiband', 18),
 ('methyl blue', 15),
 ('maternal behavior', 13),
 ('mhyle blue', 13),
 ('midbrain', 12),
 ('methyl bromide', 9),
 ('mibolerone', 9),
 ('myoglobin', 9),
 ('murabutide', 8),
 ('mammillary body', 7),
 ('midbody', 7),
 ('model based', 6),
 ('matrix bound', 6),
 ('mini b', 6),
 ('méthylène blue', 6),
 ('main branch', 5),
 ('moderator band', 5),
 ('monobutyrin', 5),
 ('methanobactin', 5),
 ('mb camp', 5),
 ('mouth breathing', 4),
 ('macranthoside b', 4),
 ('mycalolide b', 4),
 ('meldola s blue', 4),
 ('marbles buried', 3),
 ('mirror box', 3),
 ('macconkey broth', 3),
 ('membranes bound', 3),
 ('marginal band', 3),
 ('methylene blue treated', 3),
 ('methylene blue usp', 3),
 ('methylene blue dye', 3)

In [8]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [9]:
names

{'CHEBI:CHEBI:6872': 'Methylene Blue',
 'MESH:D008527': 'Medulloblastoma',
 'MESH:D024521': 'Mushroom Bodies',
 'MESH:D045423': 'Microbubbles',
 'MESH:D008344': 'Maneb',
 'MESH:D032446': 'Myoblasts',
 'MESH:D006470': 'Hemorrhage',
 'MESH:C414357': 'Methyl Blue',
 'GO:GO:0042711': 'maternal behavior',
 'MESH:D008636': 'Mesencephalon',
 'MESH:C005218': 'methyl bromide',
 'CHEBI:CHEBI:34849': 'mibolerone',
 'CHEBI:CHEBI:7044': 'myoglobin',
 'MESH:C033575': 'N-acetylmuramyl-alanylglutamine-n-butyl ester',
 'MESH:D008326': 'Mammillary Bodies',
 'GO:GO:0030496': 'midbody',
 'CHEBI:CHEBI:27560': 'boron atom',
 'CHEBI:CHEBI:76503': 'monobutyrin',
 'MESH:C491147': 'methanobactin',
 'MESH:D009058': 'Mouth Breathing',
 'MESH:C543144': 'macranthoside B',
 'MESH:C080978': 'mycalolide B',
 'GO:GO:0051748': 'UTP-monosaccharide-1-phosphate uridylyltransferase activity',
 'CHEBI:CHEBI:69120': 'withaferin A',
 'HGNC:6915': 'MB'}

In [10]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-11 18:50:26] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl
ERROR: [2020-11-11 18:52:58] adeft.gui.ground - Exception on /ground_add [POST]
Traceback (most recent call last):
  File "/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  F

In [11]:
result = [grounding_map, names, pos_labels]

In [12]:
result

[{'macconkey broth': 'ungrounded',
  'macranthoside b': 'MESH:C543144',
  'main branch': 'ungrounded',
  'major bleeding': 'ungrounded',
  'mammillary body': 'MESH:D008326',
  'maneb': 'ungrounded',
  'marbles buried': 'ungrounded',
  'marginal band': 'ungrounded',
  'maternal behavior': 'ungrounded',
  'matrix bound': 'ungrounded',
  'mb camp': 'ungrounded',
  'medulloblastoma': 'NCIT:C3222',
  'meldola s blue': 'ungrounded',
  'membranes bound': 'ungrounded',
  'methanobactin': 'MESH:C491147',
  'methyl blue': 'CHEBI:CHEBI:6872',
  'methyl bromide': 'MESH:C005218',
  'methylene blue': 'CHEBI:CHEBI:6872',
  'methylene blue dye': 'CHEBI:CHEBI:6872',
  'methylene blue treated': 'CHEBI:CHEBI:6872',
  'methylene blue usp': 'CHEBI:CHEBI:6872',
  'mhyle blue': 'ungrounded',
  'mibolerone': 'ungrounded',
  'microbubbles': 'ungrounded',
  'midbody': 'ungrounded',
  'midbrain': 'ungrounded',
  'mini b': 'ungrounded',
  'mirror box': 'ungrounded',
  'mitochondrial biogenesis': 'ungrounded',
  '

In [26]:
grounding_map, names, pos_labels = [{'macconkey broth': 'ungrounded',
  'macranthoside b': 'MESH:C543144',
  'main branch': 'ungrounded',
  'major bleeding': 'ungrounded',
  'mammillary body': 'MESH:D008326',
  'maneb': 'ungrounded',
  'marbles buried': 'ungrounded',
  'marginal band': 'ungrounded',
  'maternal behavior': 'ungrounded',
  'matrix bound': 'ungrounded',
  'mb camp': 'ungrounded',
  'medulloblastoma': 'NCIT:C3222',
  'meldola s blue': 'ungrounded',
  'membranes bound': 'ungrounded',
  'methanobactin': 'MESH:C491147',
  'methyl blue': 'CHEBI:CHEBI:6872',
  'methyl bromide': 'MESH:C005218',
  'methylene blue': 'CHEBI:CHEBI:6872',
  'methylene blue dye': 'CHEBI:CHEBI:6872',
  'methylene blue treated': 'CHEBI:CHEBI:6872',
  'methylene blue usp': 'CHEBI:CHEBI:6872',
  'mhyle blue': 'ungrounded',
  'mibolerone': 'ungrounded',
  'microbubbles': 'ungrounded',
  'midbody': 'ungrounded',
  'midbrain': 'ungrounded',
  'mini b': 'ungrounded',
  'mirror box': 'ungrounded',
  'mitochondrial biogenesis': 'ungrounded',
  'model based': 'ungrounded',
  'moderator band': 'ungrounded',
  'molecular beacon': 'ungrounded',
  'monobutyrin': 'CHEBI:CHEBI:76503',
  'mouth breathing': 'MESH:D009058',
  'mulberry extract': 'ungrounded',
  'multibacillary': 'ungrounded',
  'multiband': 'ungrounded',
  'murabutide': 'MESH:C033575',
  'musa basjoo nam wa': 'ungrounded',
  'mushroom body': 'BTO:BTO:0002675',
  'mycalolide b': 'MESH:C080978',
  'myoblasts': 'ungrounded',
  'myocardial bridge': 'ungrounded',
  'myoglobin': 'HGNC:6915',
  'méthylène blue': 'CHEBI:CHEBI:6872'},
 {'MESH:C543144': 'macranthoside B',
  'MESH:D008326': 'Mammillary Bodies',
  'NCIT:C3222': 'Medulloblastoma',
  'MESH:C491147': 'methanobactin',
  'CHEBI:CHEBI:6872': 'methylene blue',
  'MESH:C005218': 'methyl bromide',
  'CHEBI:CHEBI:76503': 'monobutyrin',
  'MESH:D009058': 'Mouth Breathing',
  'MESH:C033575': 'N-acetylmuramyl-alanylglutamine-n-butyl ester',
  'BTO:BTO:0002675': 'mushroom body',
  'MESH:C080978': 'mycalolide B',
  'HGNC:6915': 'MB'},
 ['CHEBI:CHEBI:6872', 'HGNC:6915']]

In [27]:
excluded_longforms = ['mb camp']

In [28]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [29]:
additional_entities = {'HGNC:6915': ['MB', ['MB']]}

In [30]:
unambiguous_agent_texts = {}

In [31]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [32]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [33]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [34]:
intersection1

[('HGNC:6915', 'HGNC:6915', 6308)]

In [35]:
intersection2

[('CHEBI:CHEBI:6872', 'HGNC:6915', 0),
 ('ungrounded', 'HGNC:6915', 0),
 ('MESH:D008326', 'HGNC:6915', 0),
 ('MESH:C005218', 'HGNC:6915', 0),
 ('CHEBI:CHEBI:76503', 'HGNC:6915', 0),
 ('MESH:C033575', 'HGNC:6915', 0),
 ('MESH:C080978', 'HGNC:6915', 0),
 ('BTO:BTO:0002675', 'HGNC:6915', 0),
 ('NCIT:C3222', 'HGNC:6915', 0),
 ('MESH:D009058', 'HGNC:6915', 0),
 ('MESH:C543144', 'HGNC:6915', 0),
 ('HGNC:6915', 'HGNC:6915', 3),
 ('MESH:C491147', 'HGNC:6915', 0)]

In [36]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [37]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [38]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-11 18:58:27] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-11 18:59:04] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9417929490871891 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [39]:
classifier.stats

{'label_distribution': {'CHEBI:CHEBI:6872': 686,
  'ungrounded': 294,
  'MESH:D008326': 5,
  'MESH:C005218': 7,
  'CHEBI:CHEBI:76503': 4,
  'MESH:C033575': 4,
  'MESH:C080978': 3,
  'BTO:BTO:0002675': 106,
  'NCIT:C3222': 268,
  'MESH:D009058': 2,
  'MESH:C543144': 4,
  'HGNC:6915': 259,
  'MESH:C491147': 5},
 'f1': {'mean': 0.941793, 'std': 0.018816},
 'precision': {'mean': 0.927636, 'std': 0.021701},
 'recall': {'mean': 0.958702, 'std': 0.015569},
 'CHEBI:CHEBI:6872': {'f1': {'mean': 0.932755, 'std': 0.021776},
  'pr': {'mean': 0.967915, 'std': 0.016407},
  'rc': {'mean': 0.900339, 'std': 0.0298}},
 'MESH:C491147': {'f1': {'mean': 0.4, 'std': 0.489898},
  'pr': {'mean': 0.4, 'std': 0.489898},
  'rc': {'mean': 0.4, 'std': 0.489898}},
 'ungrounded': {'f1': {'mean': 0.859318, 'std': 0.02386},
  'pr': {'mean': 0.823203, 'std': 0.039229},
  'rc': {'mean': 0.899861, 'std': 0.017131}},
 'MESH:C005218': {'f1': {'mean': 0.6, 'std': 0.489898},
  'pr': {'mean': 0.6, 'std': 0.489898},
  'rc': {'

In [40]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [41]:
disamb.dump(model_name, results_path)

In [42]:
print(disamb.info())

Disambiguation model for MB

Produces the disambiguations:
	MB*	HGNC:6915
	Mammillary Bodies	MESH:D008326
	Medulloblastoma	NCIT:C3222
	Mouth Breathing	MESH:D009058
	N-acetylmuramyl-alanylglutamine-n-butyl ester	MESH:C033575
	macranthoside B	MESH:C543144
	methanobactin	MESH:C491147
	methyl bromide	MESH:C005218
	methylene blue*	CHEBI:CHEBI:6872
	monobutyrin	CHEBI:CHEBI:76503
	mushroom body	BTO:BTO:0002675
	mycalolide B	MESH:C080978

Class level metrics:
--------------------
Grounding                                    	Count	F1     
                               methylene blue*	686	0.93276
                                   Ungrounded	294	0.85932
                              Medulloblastoma	268	 0.9604
                                           MB*	259	0.96581
                                mushroom body	106	 0.9814
                               methyl bromide	  7	    0.6
                            Mammillary Bodies	  5	    0.2
                                methanobactin	  5	    0

In [43]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1